## 豆瓣电影爬虫

In [ ]:
import json 
import time 
import random
import requests
import numpy as np
import pandas as pd
from retrying import retry
from bs4 import BeautifulSoup
from fake_useragent import UserAgent

### 基本作品爬虫

In [2]:
### 豆瓣：基本作品爬取爬虫
### 使用requests + BeautifulSoup
### 请求头使用fake_useragent
### 入口是豆瓣tag页面

#### 基础爬虫

In [3]:
donghua_tag = "https://movie.douban.com/tag/%E5%8A%A8%E7%94%BB"
def proxy_abuyun():
    # 代理服务器
    proxyHost = "http-dyn.abuyun.com"
    proxyPort = "9020"

    # 代理隧道验证信息
    proxyUser = "12345678" # 需要自己改一下
    proxyPass = "12345678" # 需要自己改一下

    # 代理页面构成
    proxyMeta = "http://%(user)s:%(pass)s@%(host)s:%(port)s" % {
        "host" : proxyHost,
        "port" : proxyPort,
        "user" : proxyUser,
        "pass" : proxyPass,
    }
    # 代理地址构造
    proxies = {
        "http"  : proxyMeta,
        "https" : proxyMeta,
    }
    return proxies

def requests_basic(url,maxpage):
    bash = 20
    url = requests.get(url)
    list_all_divs = []
    for i in range(1,maxpage):
        ua = UserAgent()
        headers = {"User-Agent":ua.random}
        start_num = i * bash
        complete_url = donghua_tag + "?start=" + str(start_num) +  "&type=T"
        print(complete_url)
        html = requests.get(complete_url,headers = headers)
        html_soup = BeautifulSoup(html.text,'lxml')
        all_divs = html_soup.find("div",class_ = "article").find_all("table",{"width":"100%"})
        list_all_divs.append(all_divs)
    return list_all_divs

def parse(list_all_divs,maxpage):
    list_hrefs,list_titles = [],[]
    for page in range(0,maxpage-1):
        print(page)
        for item in list_all_divs[page]:
            try:
                info = item.find_all("a",class_ = "nbg")
                href = info[0].get("href")
                title = info[0].get("title")
                list_hrefs.append(href)
                list_titles.append(title)
            except Exception as e:
                pass
    table_first_layer = pd.DataFrame(list_hrefs,list_titles)
    return table_first_layer

#list_all_divs = requests_basic(donghua_tag,5)
#table_first_layer = parse(list_all_divs,5)

#### 同步爬虫练习

In [4]:
# 同步爬虫练习
# coding:utf-8
# https://blog.csdn.net/feiyang123_/article/details/44700219?utm_source=blogxgwz5

import time
from lxml import etree
import urllib.request as request

# 目标网址
url = "https://movie.douban.com/tag/%E6%97%A5%E6%9C%AC?start=20&type=T"
def get_all_links(url:str)->list:
    # 获取所有的链接
    response = request.urlopen(url)
    wb_data = response.read()
    html = etree.HTML(wb_data)
    first_layer_links = html.xpath('//a[@class="nbg"]/@href')
    return first_layer_links

def get_all_titles(url:str)->list:
    # 获取所有的标题
    response = request.urlopen(url)
    wb_data = response.read()
    html = etree.HTML(wb_data)
    first_layer_titles = html.xpath('//a[@class="nbg"]/@title')
    return first_layer_titles

def generage_first_layer_table(url):
    # 获取表格
    start = time.time()
    urls = get_all_links(url)
    titles = get_all_titles(url)
    table_first_layer = pd.DataFrame(urls,titles)
    end = time.time()
    print(u"下载完成,耗时:%.2fs" % (end - start))
    return table_first_layer

def allot(first_layer_links:list,n:int)->list:
    # 根据给定的组数，分配url给每一组
    _len = len(first_layer_links)
    base = int(_len / n)
    remainer = _len % n
    groups = [first_layer_links[i * base:(i + 1) * base] for i in range(n)]
    remaind_group = first_layer_links[n * base:]
    for i in range(remainer):
        groups[i].append(remaind_group[i])
    return [i for i in groups if i]

def crawler():
    generage_first_layer_table(url)
    
#crawler()

#### 多进程爬虫练习

In [5]:
# 多进程爬虫
from multiprocessing.pool import Pool
import requests
from bs4 import BeautifulSoup

def request_single_page(url):
    # 请求单个链接
    ua = UserAgent()
    headers = {"User-Agent":ua.random}
    response = requests.get(url,headers = headers)
    html = etree.HTML(response.text)
    title = html.xpath('//span[@property="v:itemreviewed"]/text()')
    print(title)
    return title

def request_multiple_pages(urls):
    # 请求多个链接
    start = time.time()
    index = 0
    for url in urls:
        index += 1
        print(index)
        title = request_single_page(url)
        print(title)
    end = time.time()
    print(u'下载完成,耗时:%.2fs' % (end - start))

def multiprocess_crawler(processors:int,first_layer_links:list):
    # 多进程爬虫
    pool = Pool(processors)
    # 对应多进程的进程数processors,将待爬链接分组
    url_groups = allot(first_layer_links,processors)
    print(url_groups)
    for i in url_groups:
        pool.apply_async(func = request_multiple_pages,args = (i,))
    pool.close()
    pool.join()

# first_layer_links = get_all_links(url)
# request_multiple_pages(first_layer_links)
# multiprocess_crawler(2,first_layer_links)

#### 多线程爬虫练习

In [6]:
### 目前这个可用

import random
from threading import Thread

def run_multithread_crawler(first_layer_links:list,threads:int):
    begin = 0
    start = time.time()
    while 1:
        _threads = []
        urls = first_layer_links[begin:begin+threads]
        if not urls:
            break
        for i in urls:
            t = Thread(target = request_single_page,args = (i,))
            _threads.append(t)
        for t in _threads:
            t.setDaemon(True)
            t.start()
        for t in _threads:
            t.join()
        begin += threads
    end = time.time()
    print(u'下载完成,耗时:%.2fs' % ((end - start)))
    return _threads
    
def multithreads_crawler(threads:int):
    first_layer_links = get_all_links(url)
    run_multithread_crawler(first_layer_links,threads)
    
#_threads = multithreads_crawler(50)

#### 异步协程爬虫练习

In [7]:
### 目前这个也可用

import asyncio
from asyncio import Semaphore
from aiohttp import ClientSession,TCPConnector

async def download(session:ClientSession,url:str,sem:Semaphore):
    ## 请求模块
    async with sem:
        ua = UserAgent()
        headers = {"User-Agent":ua.random}
        async with session.get(url,headers = headers) as response:
            result = await response.read()
            html_soup = BeautifulSoup(result,'lxml')
            print(html_soup.find("h1"))
            
async def run_coroutine_crawler(first_layer_links:list,concurrency:int):
    # 异步协程爬虫，最大并发请求数concurrency
    tasks = []
    sem = Semaphore(concurrency)
    conn = TCPConnector(limit = concurrency)
    async with ClientSession(connector = conn) as session:
        for i in first_layer_links:
            tasks.append(asyncio.create_task(download(session,i,sem)))
        start = time.time()
        await asyncio.gather(*tasks)
        end = time.time()
        print(u'下载完成,耗时:%.2fs' % ((end - start)))
        
def coroutine_crawler(concurrency:int,url:str):
    first_layer_links = get_all_links(url)
    loop = asyncio.get_event_loop()
    loop.run_until_complete(run_coroutine_crawler(first_layer_links,concurrency))
    loop.close()

def execute():
    url = "https://movie.douban.com/tag/%E6%97%A5%E6%9C%AC?start=20&type=T"
    coroutine_crawler(100,url)

#### 多进程 + 多线程爬虫练习

In [8]:
def mixed_process_thread_crawler(first_layers_links:list,processors:int,threads:int):
    pool = Pool(processors)
    first_layer_links = get_all_links(url)
    url_groups = allot(first_layers_links,processors)
    for group in url_groups:
        pool.apply_async(run_multithread_crawler,args = (group,threads))
    pool.close()
    pool.join()

#first_layer_links = get_all_links(url)
#mixed_process_thread_crawler(first_layer_links,4,50)

#### 多进程 + 异步协程爬虫练习

In [9]:
def _coroutine_crawler(first_layers_links:list,concurrency:int):
    loop = asyncio.get_event_loop()
    loop.run_until_complete(run_coroutine_crawler(first_layers_links,concurrency))
    loop.close()

def mixed_process_coroutine_crawler(processors:int,concurrency:int):
    pool = Pool(processors)
    first_layer_links = get_all_links(url)
    url_groups = allot(pic_urls, processors)
    for group in url_groups:
        pool.apply_async(_coroutine_crawler, args=(group, concurrency))
    pool.close()
    pool.join()

#### 协程原理

In [10]:
import threading
import asyncio

async def hello():
    print('Hello world! (%s)' % threading.currentThread())
    await asyncio.sleep(5)
    print('Hello again! (%s)' % threading.currentThread())

loop = asyncio.get_event_loop()
tasks = [hello(),hello()]
loop.run_until_complete(asyncio.wait(tasks))

Hello world! (<_MainThread(MainThread, started 24448)>)
Hello world! (<_MainThread(MainThread, started 24448)>)
Hello again! (<_MainThread(MainThread, started 24448)>)
Hello again! (<_MainThread(MainThread, started 24448)>)


({<Task finished coro=<hello() done, defined at <ipython-input-10-58e59cd72d29>:4> result=None>,
  <Task finished coro=<hello() done, defined at <ipython-input-10-58e59cd72d29>:4> result=None>},
 set())

#### aiohttp构建异步爬虫框架

In [17]:
### aiohttp:requests案例
# https://zhuanlan.zhihu.com/p/36936574
# https://hubertroy.gitbooks.io/aiohttp-chinese-documentation/content/
import aiohttp
import asyncio

async def fetch(session,url):
    async with session.get(url) as response:
        return await response.text()
    
async def main():
    async with aiohttp.ClientSession() as session:
        html = await fetch(session,"http://www.baidu.com") 
        #print(html)
        
loop = asyncio.get_event_loop()
loop.run_until_complete(main())

In [30]:
# http://edmundmartin.com/concurrent-crawling-in-python/

import asyncio
import aiohttp
from bs4 import BeautifulSoup
import logging

class AsnycGrab(object):
    def __init__(self, url_list, max_threads):
        self.urls = url_list
        ### 设置results词典
        self.results = {}
        ### 设置最大线程数量
        self.max_threads = max_threads

    def _parse_results(self,url,html):
        """
        解析部分
        param url:url_list中的任一页面
        param html:url通过get_body后请求得到的结果
        return results中以字典的形式，存档title
        """
        try:
            soup = BeautifulSoup(html,'html.parser')
            title = soup.find("title").get_text()
        except Exception as e:
            raise e
        if title:
            self.results[url] = title
            
    async def get_body(self,url):
        """
        异步请求部分
        param url:url_list中的任一页面
        return html:请求后得到的status
        """
        async with aiohttp.ClientSession() as session:
            async with session.get(url,timeout = 30) as response:
                assert response.status == 200
                html = await response.read()
                return response.url,html
            
    async def get_results(self,url):
        """
        执行请求和解析两块
        """
        url,html = await self.get_body(url)
        self._parse_results(url,html)
        return 'Completed'
    
    async def handle_tasks(self, task_id, work_queue):
        ### 使用线程来分配任务！
        while not work_queue.empty():
            current_url = await work_queue.get()
            try:
                task_status = await self.get_results(current_url)
            except Exception as e:
                logging.exception('Error for {}'.format(current_url), exc_info=True)
                
    def eventloop(self):
        ### 设置任务队列！
        q = asyncio.Queue()
        [q.put_nowait(url) for url in self.urls]
        loop = asyncio.get_event_loop()
        tasks = [self.handle_tasks(task_id, q, ) for task_id in range(self.max_threads)]
        start = time.time()
        loop.run_until_complete(asyncio.wait(tasks))
        end = time.time()
        print("一共耗时" + str(end - start) + "s")

if __name__ == "__main__":
    test_urls = ["https://blog.csdn.net/jclian91/article/details/82691269"] * 5
    async_example = AsnycGrab(test_urls, 5)
    async_example.eventloop()
    print(async_example.results)

一共耗时0.8790502548217773s
{URL('https://blog.csdn.net/jclian91/article/details/82691269'): '利用aiohttp实现异步爬虫 - 但盼风雨来 - CSDN博客'}


In [31]:
### 和最简单爬虫的对比
def simplest_parse(url):
    html = requests.get(url)
    html_soup = BeautifulSoup(html.text,'lxml')
    title = html_soup.find("title").get_text()
    
test_urls = ["https://blog.csdn.net/jclian91/article/details/82691269"] * 5 
start = time.time()
for url in test_urls:
    simplest_parse(url)
end = time.time()
print("一共耗时" + str(end - start) + "s")

一共耗时2.2301275730133057s


### 基本页面信息爬虫

In [ ]:
### 豆瓣：作品基本页面信息爬虫
### 使用requests + BeautifulSoup
### 请求头使用fake_useragent
### 单线程&携程

list_soup = []
table = pd.read_excel("table_check_3.xlsx")

def proxy_abuyun():
    # 代理服务器
    proxyHost = "http-dyn.abuyun.com"
    proxyPort = "9020"

    # 代理隧道验证信息
    proxyUser = "H12AUU1SFVJ07G9D"
    proxyPass = "5F45BADF5392BD43"

    # 代理页面构成
    proxyMeta = "http://%(user)s:%(pass)s@%(host)s:%(port)s" % {
        "host" : proxyHost,
        "port" : proxyPort,
        "user" : proxyUser,
        "pass" : proxyPass,
    }
    # 代理
    proxies = {
        "http"  : proxyMeta,
        "https" : proxyMeta,
    }
    return proxies

@retry(stop_max_attempt_number=3)
def requests_single_thread(table,proxies):
    index = 0
    for item in table["url"]:
        try:
            ua = UserAgent()
            headers = {"User-Agent":ua.random}
            html = requests.get(item,headers = headers,proxies = proxies,timeout = 30)
            html_soup = BeautifulSoup(html.text,'lxml')
            list_soup.append(html_soup)
            index += 1 
            if index % 10 == 0:
                time.sleep(5)
            if index % 50 == 0:
                time.sleep(10 + random.randint(-5,5))
            print(index)
        except Exception as e:
            print("error")
            list_soup.append("error")
    return list_soup
        
def requests_coroutines(table):
    pass

proxies = proxy_abuyun()
requests_single_thread(table,proxies)

In [ ]:
list_title,list_time,list_genre = [],[],[]

def parse_to_table(list_soup,list_title,list_time,list_genre):
    for html_soup in list_soup:
        try:
            title = html_soup.find("h1").find("span",{"property":"v:itemreviewed"}).get_text()
            list_title.append(title)
        except Exception as e:
            list_title.append("None")
        try:
            time = html_soup.find("span",{"property":"v:initialReleaseDate"}).get_text()
            list_time.append(time)
        except Exception as e:
            list_time.append("None")
        try:
            genres = html_soup.find_all("span",{"property":"v:genre"})
            gen_li = []
            if genres == []:
                list_genre.append("None")
            else:
                for g in genres:
                    result = g.get_text()
                    gen_li.append(result)
                list_genre.append("，".join(gen_li))
                gen_li = []
        except Exception as e:
            list_genre.append("None")
    return list_title,list_time,list_genre

### 页面短评 + 页面评论爬虫

In [32]:
### 待完工中